In [1]:
import torch
import torchaudio
import glob
import time
import numpy as np

In [2]:
x = [
    [1,2,3,4],
    [1,2,3,4],
    [1,2,3,4]
]
x = torch.tensor(x)
print(x)
x = x.numpy()
print(x)
x = torch.tensor(x, dtype=torch.int32)
print(x)

tensor([[1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4]])
[[1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]]
tensor([[1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4]], dtype=torch.int32)


In [3]:
print(torch.randn((3,2)))
print(torch.rand((3,2)))

tensor([[ 0.7623, -1.2029],
        [ 1.5949, -0.2856],
        [ 0.2321, -1.1046]])
tensor([[0.8368, 0.0083],
        [0.1815, 0.3406],
        [0.1847, 0.5539]])


In [4]:
print(x.view(3,4,1))
print(x.unsqueeze(2))

tensor([[[1],
         [2],
         [3],
         [4]],

        [[1],
         [2],
         [3],
         [4]],

        [[1],
         [2],
         [3],
         [4]]], dtype=torch.int32)
tensor([[[1],
         [2],
         [3],
         [4]],

        [[1],
         [2],
         [3],
         [4]],

        [[1],
         [2],
         [3],
         [4]]], dtype=torch.int32)


In [5]:
x

tensor([[1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4]], dtype=torch.int32)

In [6]:
print(x.chunk(4,dim=1))
print(x.chunk(2,dim=1))

(tensor([[1],
        [1],
        [1]], dtype=torch.int32), tensor([[2],
        [2],
        [2]], dtype=torch.int32), tensor([[3],
        [3],
        [3]], dtype=torch.int32), tensor([[4],
        [4],
        [4]], dtype=torch.int32))
(tensor([[1, 2],
        [1, 2],
        [1, 2]], dtype=torch.int32), tensor([[3, 4],
        [3, 4],
        [3, 4]], dtype=torch.int32))


In [7]:
print(x.split(1,dim=1))
print(x.split(2,dim=1))

(tensor([[1],
        [1],
        [1]], dtype=torch.int32), tensor([[2],
        [2],
        [2]], dtype=torch.int32), tensor([[3],
        [3],
        [3]], dtype=torch.int32), tensor([[4],
        [4],
        [4]], dtype=torch.int32))
(tensor([[1, 2],
        [1, 2],
        [1, 2]], dtype=torch.int32), tensor([[3, 4],
        [3, 4],
        [3, 4]], dtype=torch.int32))


In [8]:
x = torch.tensor(x, dtype=torch.int64)
y = [
    [1,2,3,4]
]
y = torch.tensor(y)
z = torch.cat([x,y],dim=0)
print(x.shape)
print(y.shape)
print(z.shape)
print(z)

torch.Size([3, 4])
torch.Size([1, 4])
torch.Size([4, 4])
tensor([[1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4]])


/home/jhm9507/anaconda3/envs/jhm/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [9]:
torch.cuda.is_available()

True

In [10]:
device = torch.device("cuda")

In [11]:
z.cuda()

tensor([[1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4]], device='cuda:0')

In [12]:
z.detach()

tensor([[1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4]])

In [13]:
x = torch.randn((3,1))
y = torch.randn((1,3))
z = torch.matmul(y,x)

In [14]:
x.requires_grad_()
z.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [15]:
print(x.grad)
z = torch.matmul(y,x)
z.backward()
print(z)
print(x.grad)

None
tensor([[-0.9742]], grad_fn=<MmBackward>)
tensor([[ 0.0502],
        [ 0.5398],
        [-2.1623]])


In [16]:
from keras.datasets import mnist

Using TensorFlow backend.


In [17]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float64') / 255.0
x_test = x_test.reshape(10000, 784).astype('float64') / 255.0

x_train, y_train, x_test, y_test = map(
    torch.tensor, (x_train, y_train, x_test, y_test))

x_train, y_train, x_test, y_test = x_train.float(
), y_train.long(), x_test.float(), y_test.long()

In [18]:
from torch import nn

In [19]:
model = nn.Sequential(
    nn.Linear(784, 64),
    nn.ReLU(),
    nn.Linear(64, 10),
    nn.Softmax()
)

In [20]:
from torch import optim

In [21]:
opt = optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [22]:
epochs = 5
n = x_train.shape[0]
batch_size = 128

for epoch in range(epochs):
    for i in range(int(n/batch_size)):
        start = i*batch_size
        end = start+batch_size
        x_batch = x_train[start:end]
        y_batch = y_train[start:end]
        logit = model(x_batch)
        loss = loss_fn(logit, y_batch)
        loss.backward()
        opt.step()
        opt.zero_grad()
    logit = model(x_test)
    pred = torch.argmax(logit, dim=1)
    acc = (pred==y_test).float().mean()
    print("epoch: {}/{} loss:{} acc:{}".format(epoch,epochs,loss.detach().item(), acc.detach().item()))

/home/jhm9507/anaconda3/envs/jhm/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch: 0/5 loss:1.4858076572418213 acc:0.9150999784469604
epoch: 1/5 loss:1.4830985069274902 acc:0.926800012588501
epoch: 2/5 loss:1.4801626205444336 acc:0.9362000226974487
epoch: 3/5 loss:1.4792356491088867 acc:0.940500020980835
epoch: 4/5 loss:1.477601408958435 acc:0.9453999996185303


In [23]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)

In [24]:
class lstm(nn.Module):
    def __init__(self, input_feature, hidden_size, num_layers=1, dropout=0):
        super(lstm, self).__init__()
        self.lstm_cell = nn.LSTM(input_feature, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size, 10)
        self.softmax = nn.Softmax()
    def forward(self, x):
        output, hidden = self.lstm_cell(x)
        output = output[:,-1,:]
        output = self.relu(output)
        output = self.fc(output)
        output = self.softmax(output)
        return output

In [25]:
model = lstm(28, 128, 1, 0.3).cuda()

/home/jhm9507/anaconda3/envs/jhm/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [26]:
opt = optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [27]:
epochs = 5
n = x_train.shape[0]
batch_size = 128

x_test = x_test.cuda()
y_test = y_test.cuda()

for epoch in range(epochs):
    for i in range(int(n/batch_size)):
        start = i*batch_size
        end = start+batch_size
        x_batch = x_train[start:end].cuda()
        y_batch = y_train[start:end].cuda()
        logit = model(x_batch)
        loss = loss_fn(logit, y_batch)
        loss.backward()
        opt.step()
        opt.zero_grad()
        pred = torch.argmax(logit, dim=1)
        acc = (pred==y_batch).float().mean()
        if i%100==0:
            print("iters: {}/{} loss:{} acc:{}".format(i,int(n/batch_size),loss.detach().item(), acc.detach().item()))
    logit = model(x_test)
    pred = torch.argmax(logit, dim=1)
    acc = (pred==y_test).float().mean()
    print("epoch: {}/{} loss:{} acc:{}".format(epoch+1,epochs,loss.detach().item(), acc.detach().item()))

/home/jhm9507/anaconda3/envs/jhm/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


iters: 0/468 loss:2.3032100200653076 acc:0.1015625
iters: 100/468 loss:2.0537710189819336 acc:0.4609375
iters: 200/468 loss:1.7979481220245361 acc:0.6953125
iters: 300/468 loss:1.7168775796890259 acc:0.7421875
iters: 400/468 loss:1.7549362182617188 acc:0.703125
epoch: 1/5 loss:1.6289225816726685 acc:0.7627999782562256
iters: 0/468 loss:1.6300321817398071 acc:0.828125
iters: 100/468 loss:1.6267740726470947 acc:0.828125
iters: 200/468 loss:1.6508917808532715 acc:0.8203125
iters: 300/468 loss:1.5930616855621338 acc:0.8671875
iters: 400/468 loss:1.5976910591125488 acc:0.859375
epoch: 2/5 loss:1.4879093170166016 acc:0.9073999524116516
iters: 0/468 loss:1.5333691835403442 acc:0.9375
iters: 100/468 loss:1.5377756357192993 acc:0.9296875
iters: 200/468 loss:1.5054078102111816 acc:0.96875
iters: 300/468 loss:1.5055662393569946 acc:0.953125
iters: 400/468 loss:1.521694540977478 acc:0.953125
epoch: 3/5 loss:1.468802571296692 acc:0.9521999955177307
iters: 0/468 loss:1.4822744131088257 acc:0.984375


In [30]:
mat1 = torch.randn((4,20,128))
mat2 = torch.randn((4,128,10))

torch.bmm(mat1, mat2).shape

torch.Size([4, 20, 10])

In [22]:
x = torch.ones((4,20,80))
y = torch.zeros((4,20,80))
z = torch.cat((x,y),dim=2)

In [23]:
z

tensor([[[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 

In [24]:
z.view(4,-1)

tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]])

In [25]:
c = z.transpose(0,1)

In [26]:
c.is_contiguous()

False

In [27]:
c = c.contiguous()

In [28]:
c.is_contiguous()

True

In [29]:
x = torch.rand(5)

In [30]:
x

tensor([0.4694, 0.7972, 0.5963, 0.7721, 0.1355])

In [31]:
x.topk(1)

torch.return_types.topk(
values=tensor([0.7972]),
indices=tensor([1]))

In [32]:
x.data.eq(1)

tensor([False, False, False, False, False])

In [33]:
x.eq(1)

tensor([False, False, False, False, False])

In [34]:
x.detach().eq(1)

tensor([False, False, False, False, False])

In [5]:
feature_size = 512
model = torch.nn.Transformer()

In [ ]:
torch.nn.TransformerEncoderLayer()